In [ ]:
import sys
import os

# Add the parent directory to Python path so we can import modules from the project root
# In Jupyter notebooks, __file__ is not defined, so we use getcwd() and navigate up
current_dir = os.getcwd()
# If we're in the testing directory, go up one level to get to project root
if current_dir.endswith('/testing'):
    project_root = os.path.dirname(current_dir)
else:
    # If we're already in project root, use current directory
    project_root = current_dir

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from bn_helpers.get_structures_print_tools import get_nets, printNet
from bn_helpers.constants import MODEL, MODEL_QUIZ
from benchmarking.question_types import (DEPENDENCY_QUESTIONS, COMMON_CAUSE_QUESTIONS, COMMON_EFFECT_QUESTIONS, BLOCKED_EVIDENCES_QUESTIONS, 
EVIDENCE_CHANGE_RELATIONSHIP_QUESTIONS, PROBABILITY_QUESTIONS, HIGHEST_IMPACT_EVIDENCE_QUESTIONS)
from benchmarking.model_evaluator import (dependency_test, common_cause_test, common_effect_test, blocked_evidence_test, 
evidence_change_relationship_test, probability_test, highest_impact_evidence_test)
from benchmarking.data_utils import load_nets_from_parquet
import asyncio

data_output = os.path.join(project_root, "benchmarking", "data")
print(f"Loading nets from: {data_output}")
net_5, net_10, net_30, net_60 = load_nets_from_parquet(os.path.join(data_output, "nets_dataset.parquet"))

list_of_nets = [net_5, net_10, net_30, net_60]
NUM_QUESTIONS = 30
MAX_TOKENS = 1800
IS_TESTING = True
PROBABILITY_MAX_TOKENS = 300

Loading nets from: /fs04/scratch2/lb64/projects/llm-bn/benchmarking/data
Loaded 4 nets from /fs04/scratch2/lb64/projects/llm-bn/benchmarking/data/nets_dataset.parquet


In [ ]:
import time
import random
from pydantic_ai.exceptions import ModelHTTPError

def retry_test_with_backoff(test_function, net, max_retries=5, base_delay=1, max_delay=60, *args, **kwargs):
    """
    Retry a test function with exponential backoff when encountering HTTP errors.
    
    Args:
        test_function: The test function to retry
        net: The network parameter
        max_retries: Maximum number of retry attempts
        base_delay: Base delay in seconds for exponential backoff
        max_delay: Maximum delay in seconds
        *args, **kwargs: Additional arguments to pass to the test function
    
    Returns:
        The result of the test function or raises the last exception if all retries fail
    """
    last_exception = None
    
    for attempt in range(max_retries + 1):
        try:
            print(f"Attempt {attempt + 1}/{max_retries + 1} for {test_function.__name__}")
            result = test_function(net, *args, **kwargs)
            print(f"✅ {test_function.__name__} completed successfully on attempt {attempt + 1}")
            return result
            
        except ModelHTTPError as e:
            last_exception = e
            if e.status_code == 500:  # Internal Server Error
                if attempt < max_retries:
                    # Calculate delay with exponential backoff and jitter
                    delay = min(base_delay * (2 ** attempt) + random.uniform(0, 1), max_delay)
                    print(f"❌ HTTP 500 error on attempt {attempt + 1}. Retrying in {delay:.2f} seconds...")
                    time.sleep(delay)
                else:
                    print(f"❌ All {max_retries + 1} attempts failed for {test_function.__name__}")
                    break
            else:
                # For non-500 errors, don't retry
                print(f"❌ Non-retryable HTTP error {e.status_code} for {test_function.__name__}")
                raise e
                
        except Exception as e:
            last_exception = e
            print(f"❌ Unexpected error on attempt {attempt + 1}: {str(e)}")
            if attempt < max_retries:
                delay = min(base_delay * (2 ** attempt) + random.uniform(0, 1), max_delay)
                print(f"Retrying in {delay:.2f} seconds...")
                time.sleep(delay)
            else:
                print(f"❌ All {max_retries + 1} attempts failed for {test_function.__name__}")
                break
    
    # If we get here, all retries failed
    print(f"🚨 {test_function.__name__} failed after {max_retries + 1} attempts")
    raise last_exception


In [ ]:
DEEPSEEK_MODEL = "deepseek-r1:8b"
LLAMA_MODEL = "llama3.1:70b"
MODEL_LIST = [DEEPSEEK_MODEL, LLAMA_MODEL]
for MODEL in MODEL_LIST:
    dependency_test(net_5, num_questions=NUM_QUESTIONS, max_tokens=MAX_TOKENS, isTesting=IS_TESTING, model=MODEL)
    common_cause_test(net_5, num_questions=NUM_QUESTIONS, max_tokens=MAX_TOKENS, isTesting=IS_TESTING, model=MODEL)
    common_effect_test(net_5, num_questions=NUM_QUESTIONS, max_tokens=MAX_TOKENS, isTesting=IS_TESTING, model=MODEL)
    blocked_evidence_test(net_5, num_questions=NUM_QUESTIONS, max_tokens=MAX_TOKENS, isTesting=IS_TESTING, model=MODEL)
    evidence_change_relationship_test(net_5, num_questions=NUM_QUESTIONS, max_tokens=MAX_TOKENS, isTesting=IS_TESTING, model=MODEL)
    try:
        retry_test_with_backoff(probability_test, net_5, max_retries=5, base_delay=2, max_delay=30,\
            num_questions=NUM_QUESTIONS, max_tokens=PROBABILITY_MAX_TOKENS, isTesting=IS_TESTING, model=MODEL)
    except Exception as e:
        print(f"probability_test failed completely: {str(e)}")
        print("Continuing with next test...")

    print(f"\nCompleted all tests for Net_{len(net_5.nodes())}")

Running dependency test for Net_5 with 30 questions
Found 30 already completed questions: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Skipping completed questions and continuing from where we left off...
Skipping question 1 (already completed)
Skipping question 2 (already completed)
Skipping question 3 (already completed)
Skipping question 4 (already completed)
Skipping question 5 (already completed)
Skipping question 6 (already completed)
Skipping question 7 (already completed)
Skipping question 8 (already completed)
Skipping question 9 (already completed)
Skipping question 10 (already completed)
Skipping question 11 (already completed)
Skipping question 12 (already completed)
Skipping question 13 (already completed)
Skipping question 14 (already completed)
Skipping question 15 (already completed)
Skipping question 16 (already completed)
Skipping question 17 (already completed)
Skipping question 18 (already completed)

KeyboardInterrupt: 

In [ ]:
# Run tests with retry mechanism for probability_test and highest_impact_evidence_test
for net in list_of_nets:
    print(f"\n{'='*60}")
    print(f"Running tests for Net_{len(net.nodes())}")
    print(f"{'='*60}")
    
    # Run the first 4 tests normally (they seem to work fine)
    dependency_test(net, num_questions=NUM_QUESTIONS, max_tokens=MAX_TOKENS, isTesting=IS_TESTING)
    common_cause_test(net, num_questions=NUM_QUESTIONS, max_tokens=MAX_TOKENS, isTesting=IS_TESTING)
    common_effect_test(net, num_questions=NUM_QUESTIONS, max_tokens=MAX_TOKENS, isTesting=IS_TESTING)
    blocked_evidence_test(net, num_questions=NUM_QUESTIONS, max_tokens=MAX_TOKENS, isTesting=IS_TESTING)
    evidence_change_relationship_test(net, num_questions=NUM_QUESTIONS, max_tokens=MAX_TOKENS, isTesting=IS_TESTING)
    
    # Run the problematic tests with retry mechanism
    print(f"\n🔄 Starting probability_test with retry mechanism...")
    try:
        retry_test_with_backoff(
            probability_test, 
            net, 
            max_retries=5, 
            base_delay=2, 
            max_delay=30,
            num_questions=NUM_QUESTIONS, 
            max_tokens=PROBABILITY_MAX_TOKENS, 
            isTesting=IS_TESTING
        )
    except Exception as e:
        print(f"🚨 probability_test failed completely: {str(e)}")
        print("Continuing with next test...")
    
    print(f"\n🔄 Starting highest_impact_evidence_test with retry mechanism...")
    try:
        retry_test_with_backoff(
            highest_impact_evidence_test, 
            net, 
            max_retries=5, 
            base_delay=2, 
            max_delay=30,
            num_questions=NUM_QUESTIONS, 
            max_tokens=PROBABILITY_MAX_TOKENS, 
            isTesting=IS_TESTING
        )
    except Exception as e:
        print(f"🚨 highest_impact_evidence_test failed completely: {str(e)}")
        print("Continuing with next network...")
    
    print(f"\n✅ Completed all tests for Net_{len(net.nodes())}")

print(f"\n🎉 All networks processed!")
